In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

from tqdm import tqdm

/home/msst/Utils/miniconda3/envs/qenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# FP
path_to_model = "/media/msst/ssd_storage1/ml/llm/pretrained_models/Llama2-7B"
tokenizer = AutoTokenizer.from_pretrained(path_to_model)
# model = AutoModelForCausalLM.from_pretrained(
model = qlib._modeling.modeling_llama.LlamaForCausalLM.from_pretrained(
    path_to_model,
    device_map="cpu",
    dtype="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 109.09it/s]


In [3]:
class QuantLinear(nn.Module):
    def __init__(self, weight_shape, group_size):
        super().__init__()
        self.weight_shape = weight_shape
        self.group_size = group_size
        self.scale_size = [weight_shape[0], weight_shape[1] // group_size]
        self.scale = nn.Parameter(torch.empty(self.scale_size))
        self.offset = nn.Parameter(torch.empty(self.scale_size))
        self.register_buffer(
            "compressed_weight",
            torch.empty(
                weight_shape,
                dtype=torch.uint8,
                requires_grad=False
            )
        )

    def reshape_weight_for_scaling(self, w):
        return w.reshape(
            self.weight_shape[0], self.weight_shape[1] // self.group_size, -1
        )


    @torch.compile()
    def reconstruct_weight(self):
        w = self.reshape_weight_for_scaling(self.compressed_weight)
        w = w * self.scale[..., None] - self.offset[..., None]
        w = w.reshape(self.weight_shape)
        return w


    def forward(self, x):
        w = self.reconstruct_weight()
        return torch.nn.functional.linear(x, w)

In [4]:
qmodel = AutoModelForCausalLM.from_pretrained(
    path_to_model,
    device_map="cpu",
    dtype="auto"
)

def wrap_model(current_module, prefix=''):
    for module_name, module in current_module.named_children():
        full_name = f"{prefix}.{module_name}" if prefix else module_name
        
        if "proj" in module_name:
            weight_shape = module.weight.data.shape
            setattr(current_module, module_name, QuantLinear(weight_shape, 128))
        else:
            wrap_model(module, full_name)

wrap_model(qmodel)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 105.31it/s]


In [5]:
class MinMaxInitializer:
    def __init__(self):
        pass

    @torch.no_grad()
    def __call__(self, x_grouped, negative_clip, positive_clip):
        x_min = x_grouped.min(axis=-1)[0].unsqueeze(-1).float()
        x_max = x_grouped.max(axis=-1)[0].unsqueeze(-1).float()

        offset = (x_max * negative_clip - x_min * positive_clip) / (positive_clip - negative_clip)
        scale = (x_max + offset) / positive_clip
        scale = torch.abs(scale)

        scale = scale.reshape(x_grouped.shape[0], x_grouped.shape[1])
        offset = offset.reshape(x_grouped.shape[0], x_grouped.shape[1])

        return scale.contiguous(), offset.contiguous()

initializer = MinMaxInitializer()

@torch.no_grad()
def configure_single_layer(qlayer, layer, bits):
    max_int_val = 2**bits - 1

    orig_weight = layer.weight.data
    orig_weight_reshaped = qlayer.reshape_weight_for_scaling(orig_weight)
    scale, offset = initializer(orig_weight_reshaped, negative_clip=0, positive_clip=max_int_val)
    
    quant_weight = (orig_weight_reshaped + offset[..., None]) / scale[..., None]
    quant_weight = quant_weight.reshape_as(orig_weight)

    quant_weight = torch.clamp(torch.round(quant_weight), 0, max_int_val).to(torch.uint8)

    qlayer.compressed_weight.copy_(quant_weight)
    qlayer.scale.copy_(scale)
    qlayer.offset.copy_(offset)


@torch.no_grad()
def init_quant_model(qmodel, model, bits):
    for qmodule_name, qmodule in qmodel.named_modules():
        if isinstance(qmodule, QuantLinear):
            orig_module = model.get_submodule(qmodule_name)
            configure_single_layer(qmodule, orig_module, bits)

            err = torch.mean(((orig_module.weight.data.cpu() - qmodule.reconstruct_weight().cpu()) / (orig_module.weight.data.cpu().std() + 1e-8))**2)
            print(err, qmodule_name)


# init_quant_model(qmodel, model, bits=3)

In [6]:
from transformers.masking_utils import create_causal_mask


@torch.no_grad()
def prepare_hessian(activations):
    hidden_size = activations[0].shape[-1]
    H = torch.zeros(hidden_size, hidden_size).cuda()
    for act in activations:
        act = act.cuda().view(-1, act.shape[-1]).float() / hidden_size ** 2
        H += act.T @ act
    return H


def hessian_loss(layer_q, layer_fp, H):
    delta_w = layer_q.reconstruct_weight() - layer_fp.weight
    return torch.trace(delta_w @ H @ delta_w.T)


def optimize_quant_params(
        layer_q,
        layer_fp,
        bits,
        H
    ):
    trainable_params = [layer_q.scale, layer_q.offset]
    # optim = torch.optim.Adam(trainable_params, lr=3e-4)
    # n_steps = 100    
    
    optim = torch.optim.Adam(trainable_params, lr=1e-3)
    n_steps = 100
    
    with torch.amp.autocast('cuda', dtype=torch.bfloat16):
        for i in range(n_steps):
            optim.zero_grad()
            loss = hessian_loss(layer_q, layer_fp, H)
            if i == 0:
                init_loss =  loss.item()
            loss.backward()
            optim.step()

        print(f"{init_loss} -> {loss}")


def init_quant_block_hessian(
        block_q,
        block_fp,
        bits,
        activations,
        causal_mask,
        position_embeddings
        ):

    ##### Attention #####

    # Copy activations for the residual stream
    residual_activations = [x.clone() for x in activations]

    # Collect activations after input_layernorm
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp.input_layernorm(act)
            activations[act_id] = act.cpu()

    # Initialize q,k,v-projs
    H = prepare_hessian(activations)
    block_q_attn = block_q.self_attn
    block_fp_attn = block_fp.self_attn
    for layer_name in ["q_proj", "k_proj", "v_proj"]:
        layer_q = getattr(block_q_attn, layer_name)
        layer_fp = getattr(block_fp_attn, layer_name)
        configure_single_layer(layer_q, layer_fp, bits)
        optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect attention-out activations
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp_attn.compute_attention(
                hidden_states=act, 
                position_embeddings=position_embeddings,
                attention_mask=causal_mask
            )[0]
            activations[act_id] = act.cpu()

    # Initialize o_proj
    layer_q = block_q.self_attn.o_proj
    layer_fp = block_fp.self_attn.o_proj
    H = prepare_hessian(activations)
    configure_single_layer(layer_q, layer_fp, bits)
    optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect self_attn outs
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            res_act = residual_activations[act_id].cuda()
            act = block_fp.self_attn.o_proj(act)
            activations[act_id] = (act + res_act).cpu()

    ##### MLP #####

    # Copy activations for the residual stream
    residual_activations = [x.clone() for x in activations]

    # Collect activations after post_attention_layernorm
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp.post_attention_layernorm(act)
            activations[act_id] = act.cpu()

    # Initialize gate_proj and up_proj
    H = prepare_hessian(activations)
    block_q_mlp = block_q.mlp
    block_fp_mlp = block_fp.mlp
    for layer_name in ["gate_proj", "up_proj"]:
        layer_q = getattr(block_q_mlp, layer_name)
        layer_fp = getattr(block_fp_mlp, layer_name)
        configure_single_layer(layer_q, layer_fp, bits)
        optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect internal mlp activations
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            act = block_fp_mlp.act_fn(block_fp.mlp.gate_proj(act)) * block_fp.mlp.up_proj(act)
            activations[act_id] = act.cpu()

    # Initialize down_proj
    layer_q = block_q.mlp.down_proj
    layer_fp = block_fp.mlp.down_proj
    H = prepare_hessian(activations)
    configure_single_layer(layer_q, layer_fp, bits)
    optimize_quant_params(layer_q, layer_fp, bits, H)

    # Collect mlp outs
    with torch.no_grad():
        for act_id, act in enumerate(activations):
            act = act.cuda()
            res_act = residual_activations[act_id].cuda()
            act = block_fp.mlp.down_proj(act)
            activations[act_id] = (act + res_act).cpu()


def init_quant_model_hessian(model_q, model_fp, bits, dataloader):
    embed_tokens = model_fp.get_decoder().embed_tokens.cuda()
    embed_tokens_device = embed_tokens.weight.device

    _batch = next(iter(dataloader))
    _inputs_embeds = embed_tokens(_batch.to(embed_tokens_device))

    cache_position = torch.arange(_inputs_embeds.shape[1], device=_inputs_embeds.device)
    position_ids = cache_position.unsqueeze(0)
    causal_mask = create_causal_mask(
        config=model_fp.config,
        input_embeds=_inputs_embeds,
        attention_mask=None,
        cache_position=cache_position,
        past_key_values=None,
        position_ids=position_ids,
    )

    position_embeddings = model_fp.get_decoder().rotary_emb(_inputs_embeds, position_ids)

    # Prepare activations
    activations = []
    with torch.no_grad():
        for batch in dataloader:
            activations.append(embed_tokens(batch.to(embed_tokens_device)).cpu())

    for decoder_layer_id in tqdm(range(len(model_q.get_decoder().layers))):
        block_q = model_q.get_decoder().layers[decoder_layer_id].cuda()
        block_fp = model_fp.get_decoder().layers[decoder_layer_id].cuda()
        
        init_quant_block_hessian(
            block_q,
            block_fp,
            bits,
            activations,
            causal_mask,
            position_embeddings
        )

        block_q = block_q.cpu()
        block_fp = block_fp.cpu()


config = {
    "dataset_name" : "wiki",
    "split": "train[:7500]",
    "seq_length": 4096,
    "n_seq" : 64,
    "batch_size": 8,
    "random_seed": 'no_rand'
}
dataloader = qlib.QATDataset(
    config=config,
    tokenizer=tokenizer
).get_dataloader()
print(len(dataloader))

init_quant_model_hessian(qmodel, model, 3, dataloader)

8


  0%|          | 0/32 [00:00<?, ?it/s]

init_loss: tensor(1.2502e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(4.2263e-10, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.3721e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(4.5248e-10, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(3.8072e-10, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(1.3621e-10, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(5.2383e-12, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(4.9861e-12, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(9.7145e-10, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(8.0465e-10, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(9.3452e-10, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(7.6174e-10, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.9322e-13, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(1.9322e-13, device='cuda:0', grad_fn=<Tr

  3%|▎         | 1/32 [00:31<16:19, 31.60s/it]

init_loss: tensor(1.4616e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(3.7970e-09, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.3599e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(3.8350e-09, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(2.0242e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(5.1427e-10, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(2.5040e-11, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(2.3637e-11, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(3.2190e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(2.9534e-09, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(2.8283e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(2.5901e-09, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(7.2292e-10, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(6.1293e-12, device='cuda:0', grad_fn=<Tr

...

 94%|█████████▍| 30/32 [15:27<01:00, 30.42s/it]

init_loss: tensor(9.6619e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(8.2085e-08, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(9.5392e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(8.0846e-08, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(8.6496e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(7.3428e-08, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(7.6393e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(6.3326e-09, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.5134e-07, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(1.3055e-07, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.3521e-07, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(1.1682e-07, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.9375e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(1.0397e-09, device='cuda:0', grad_fn=<Tr

 97%|█████████▋| 31/32 [15:58<00:30, 30.40s/it]

init_loss: tensor(6.6953e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(5.7151e-08, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(7.0954e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(6.0492e-08, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(4.9710e-08, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(4.2320e-08, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(8.7753e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(6.9127e-09, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.3666e-07, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(1.1600e-07, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(1.2210e-07, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(1.0359e-07, device='cuda:0', grad_fn=<TraceBackward0>)
init_loss: tensor(6.2121e-09, device='cuda:0', grad_fn=<TraceBackward0>)
res_loss: tensor(3.5146e-09, device='cuda:0', grad_fn=<Tr

100%|██████████| 32/32 [16:28<00:00, 30.89s/it]


In [7]:
config = {
    "dataset_name" : "wiki",
    "split": "test",
    "seq_length": 4096,
    "batch_size": 1,
    "random_seed": 'no_rand'
}

dataloader = qlib.QATDataset(
    config=config,
    tokenizer=tokenizer
).get_dataloader()

qmodel = qmodel.cuda()
# qmodel = model.cuda()

with torch.no_grad():
    with torch.amp.autocast('cuda', dtype=torch.float16):
        res = qlib.evaluate(qmodel, dataloader, print_times=25)
        print(res)

  5%|▍         | 4/83 [00:04<01:32,  1.17s/it]

5.326810313460885


  8%|▊         | 7/83 [00:07<01:29,  1.18s/it]

5.49138789832124


 12%|█▏        | 10/83 [00:10<01:25,  1.18s/it]

...

5.771935874904906


 99%|█████████▉| 82/83 [01:25<00:01,  1.18s/it]

5.8045090858022625


100%|██████████| 83/83 [01:25<00:00,  1.03s/it]


5.814998848786007
